# TP Relation Extraction

## Introduction

Dans ce TP nous allons:

1. Découvrir la tache d’extraction de relation par réseaux neuronaux  (« neural relation extraction », NRE)
2.  Comprendre le fonctionnement d’un modèle d’extraction de relation avec un encodeur BERT
3. Découvrir la tache de reconnaissance d'entités nommées (« named-entity recognition », NER)
4. Coder une pipeline d’extraction de relation par réseaux neuronaux pour des jeux de données textuels

Avec les outils suivants:
1. La libraire [OpenNRE](https://github.com/thunlp/OpenNRE), basée sur Pytorch et HuggingFace’s Transformers, pour la tache d’extraction de relation par réseaux neuronaux
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Contrairement au dernier TP, cette fois-ci nous n’aurons pas besoin d’utiliser une GPU car nous n’allons pas entraîner des nouveaux modèles ou faire de l’inférence sur des grands jeux de données: une CPU suffira. Nous pouvons vérifier si l’on est en train d’utiliser une CPU ou une GPU avec les lignes suivantes:

In [1]:
import torch

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

Will work on CPU.


Nous avons besoin d’installer l’outil OpenNRE. Pour éviter de devoir ré-télécharger le répertoire  GitHub de OpenNRE à chaque fois qu’on ré-initialise le fichier Colab, il est convenant de monter notre répertoire Google Drive et y télécharger le répertoire OpenNRE de façon à l’avoir toujours disponible. Ainsi, à chaque fois que nous allons ré-initialiser le fichier Colab, il nous suffira de monter notre Goodle Drive pour avoir accès à la libraire OpenNRE.

> ATTENTION: modifiez *tp_path_in_drive* pour pointer vers le repertoire où vous avez placé le fichier tp_re.ipynb, vous allez télécharger OpenNRE dans le meme réépertoire. Si vous êtes sur votre machine locale, vous n'avez pas besoin de monter le Drive, mais juste de faire le clonage du réépertoire OpenNRE.

In [2]:
import os
from google.colab import drive

tp_path_in_drive = '/content/gdrive/My Drive/tp_entitéNommée'
opennre_path_in_drive = tp_path_in_drive + '/OpenNRE'

# mount Google Drive
drive.mount('/content/gdrive')

if not os.path.isdir(opennre_path_in_drive):
  # OpenNRE is not already present in Google Drive
  if not os.path.isdir(tp_path_in_drive):
    # make directory for the TP if necessary
    os.makedirs(tp_path_in_drive, exist_ok=True)
  # change directory to the TP directory
  os.chdir(tp_path_in_drive)
  # clone OpenNRE repo
  print("Cloning repo...")
  os.system('git clone https://github.com/thunlp/OpenNRE.git')
  print("...done!")
else:
  print("OpenNRE is already present in Google Drive under {0}".format(opennre_path_in_drive))

# Change current dir to OpenNRE
os.chdir(opennre_path_in_drive)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
OpenNRE is already present in Google Drive under /content/gdrive/My Drive/tp_entitéNommée/OpenNRE


In [3]:
# Update requirements
!sed -i '/transformers==3.0.2/c\transformers==3.4.0' requirements.txt

Nous pouvons désormais continuer avec l’installation.

In [4]:
!pip install -r requirements.txt
!python setup.py install

running install
running bdist_egg
running egg_info
writing opennre.egg-info/PKG-INFO
writing dependency_links to opennre.egg-info/dependency_links.txt
writing top-level names to opennre.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'opennre.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/opennre
copying build/lib/opennre/__init__.py -> build/bdist.linux-x86_64/egg/opennre
copying build/lib/opennre/pretrain.py -> build/bdist.linux-x86_64/egg/opennre
creating build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/__init__.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/base_encoder.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/bert_encoder.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encod

## Les modéles pour l'extraction de relation

À part offrir un cadre pour l’implémentation et l’entraînement de modelés pour l’extraction de relation, OpenNRE offre aussi des modèles déjà entraînés sur différents jeux de données, et donc capables de détecter différents types de relations entre entités. 

Ici, nous allons employer un modèle entraîné sur Wiki80 (dataset introduit par [le papier OpenNRE](https://www.aclweb.org/anthology/D19-3029/)), un jeu de données contenant des phrases collectées sur Wikipedia et Wikidata, ainsi que des rélations entre leurs entités. Si vous voulez en savoir plus sur Wiki80, vous pouvez le télécharger avec le script [download_wiki80.sh](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/benchmark/download_wiki80.sh) 

In [5]:
import opennre
model = opennre.get_model('wiki80_bert_softmax')

2022-03-14 10:54:55,976 - root - INFO - Loading BERT pre-trained checkpoint.


Nous pouvons utiliser ce modèle pour calculer la relation entre un mot «tête» et un mot «queue» qui sont contenus dans un texte. Il suffit de passer au modèle le texte ainsi que la position de la tête et de la queue. Le modèle retournera la relation de la queue à l’égard de la tête, ainsi que la probabilité qu’il associe à cette rélation. Par example, nous pouvons inféérer la relation entre **Áed Uaridnach** et **Máel Dúin mac Máele Fithrich** de la façon suivante:

In [6]:
item = {'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (78, 91)}, 't': {'pos': (18, 46)}}
print(model.infer(item))

('child', 0.9812852144241333)


#### Exercice 1



Écrire une fonction `to_input_format(text, head, tail)` qui nous permet de trouver la position de deux mots (une tête et une queue) dans un texte, et qui retourne un dictionnaire contenant le texte et les deux positions suivant le format requis par la fonction `model.infer()`.

In [7]:
import re

def to_input_format(text, head, tail):
  """
  Args:
    text: a string of text
    head: a string of text representing a word contained in text
    tail: a string of text representing a word contained in text but different from head
  Returns:
    A dictionary containing the text and the position of the head and tail within it,
    following the input format required by an OpenNRE model (see exemple above).
  """
  head_pos_begin = text.find(head)
  head_pos_end = head_pos_begin + len(head)

  tail_pos_begin = text.find(tail)
  tail_pos_end = tail_pos_begin + len(tail)

  return {'text' : text, 'h' : {'pos' : (head_pos_begin, head_pos_end)}, 't': {'pos': (tail_pos_begin, tail_pos_end)}}
  # ....

In [8]:
# Test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
head = 'Áed Uaridnach'
tail = 'Máel Dúin mac Máele Fithrich'
test_item = to_input_format(text, head, tail)
try:
  assert model.infer(item) == model.infer(test_item)
  print("Good job!")
except AssertionError:
  print("Something is wrong with your function, try again!")

Good job!


In [9]:
print(model)

SoftmaxNN(
  (sentence_encoder): BERTEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
        

#### Exercice 2

Décrire l’architecture du modèle qu’on vient de télécharger, la logique et le fonctionnement de chacun de ses composants:

1. Sentence-encoder: pourquoi l'on utilise un encodeur type BERT ?
2. À quoi sert la couche BertPooler ?
3. À quoi sert la couche linéaire finale (fc) ? pourquoi elle réduit à 80 la dimension du vecteur sortant de l’encodeur?
4. À quoi sert la fonction Softmax ?
5. À quoi sert la fonction de dropout que l'on appliuque à la sortie du réseau, ainsi que dans chaque couche de BERT ?

Aide: vous pouvez inspecter un modèle Pytorch avec `print(model)`. Pour mieux le comprendre, vous pouvez aussi voir [son code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5), [la déscription du modèle BERT de Huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/bert#transformers.BertModel) ainsi que [son code](https://github.com/huggingface/transformers/blob/v4.16.2/src/transformers/models/bert/modeling_bert.py#L848).

### Réponses 

1. On utilise un encoder type BERT car on réalise une sorte de classification (on cherche à prédire la relation la plus probable entre deux entités nommées). **Pourquoi BERT est le modèle de choix pour cette tache? -> Car il est capable de représenter la signification de chaque mot en fonction des mots qui l'entourent dans la même phrase (représentation contextualisée), c'est qui est très utile pour une comprehension générale de la phrase et sa classification. D'ailleurs, les représentations contextualisées sortant de BERT peuvent être utilisées pour beaucoup d'autres taches, et non-seulement la multi-classification**
2. La couche BertPooler permet de regrouper la sortie de l’encodeur BERT avant la couche finale. BertPooler permet de garder seulement la représentation cachée du premier token qui résume le texte.
3. La couche linéaire finale permet de réduire la dimension du vecteur sortant de l'encodeur à 80 car on a 80 classes, c'est-à-dire 80 relations possibles.
4. La fonction Softmax permet de normaliser et d'obtenir un vecteur de probabilité de taille 80 (pour avoir la probabilité d'avoir chacune des classes/des relations).
5. La fonction dropout que l'on applique à la sortie du réseau, ainsi que dans chaque couche de BERT permet de sélectionner qu'une partie des neurones du réseau pour éviter l'overfitting. **Pourquoi le dropout permet d'éviter le overfitting?**


## Encodeur

#### Exercice 3

Maintenant, nous allons essayer de mieux comprendre le fonctionnement de ce modèle avec un focus sur son encodeur, qui est définit dans le fichier [bert_encoder.py](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/encoder/bert_encoder.py#L7). Ce qui est spécifique à la tache de NRE dans ce modèle n’est pas l’architecture, mais plutôt la façon dont la séquence en input est tokenisée. Avec l’objectif de bien comprendre comment cet encodeur gère son input, répondez en détail aux questions suivantes qui se référent à la méthode `tokenize`. Si vous utilisez du code pour vous aider à répondre (conseillé), veuillez le joindre à vos réponses textuelles.

1. Qu’est-ce qu’elle sont les variables `sent0`, `ent0`, `sent1`, `ent1`, `sent2` ?
2. Qu’est-ce que c’est `re_tokens` ?
3. Dans le _forward_, le `BERTEncoder` prend en entrée exclusivement la séquence textuelle qui a étée préalablement tokenisée. Comment est-il capable de distinguer la tête et la queue en sort de (apprendre à) prédire la relation entre les deux ?
4. Qu’est-ce que c’est le « Padding » et pourquoi est-il utile ?
5. Qu’est-ce que c’est l’ « Attention mask » et pourquoi est-elle utile ? [Aide ici](https://huggingface.co/docs/transformers/v4.16.2/en/glossary#attention-mask)
6. **[BONUS]** Quelle est la différence entre la classe `BERTEncoder` et la classe `BERTEntityEncoder`, définie dans le même fichier ?


Aide : vous pouvez accéder à la méthode `tokenize` pour la tester avec `model.sentence_encoder.tokenize(item)`

In [10]:
# Write your code here if needed, and then write your detailed answers below.
tokenize = model.sentence_encoder.tokenize(item)
model.sentence_encoder.forward(tokenize[0], tokenize[1], tokenize[2], tokenize[3])

tensor([[ 0.4638, -0.4593,  0.2540, -0.5908, -0.0125,  0.6642,  0.9365,  0.0483,
         -0.7273, -0.3425,  0.8914,  0.7780,  0.4055, -0.8910,  0.9383, -0.1310,
         -0.8897,  0.8446, -0.6228, -0.7706,  0.3430,  0.8529,  0.0679,  0.4727,
          0.6341, -0.9478,  0.8847,  0.8844,  0.1530, -0.6425,  0.7174, -0.7653,
         -0.9471, -0.3960, -0.6971, -0.3486,  0.8312, -0.7198,  0.2090,  0.5294,
         -0.8037,  0.1047,  0.5383, -0.5701, -0.6731, -0.0634,  0.9902, -0.8422,
          0.8534, -0.8804, -0.9516, -0.0551,  0.1833,  0.5383,  0.2994,  0.1594,
          0.5757, -0.4190, -0.0261,  0.7237, -0.5949,  0.7558,  0.8197,  0.2948,
         -0.2185,  0.1810, -0.5062, -0.7662,  0.6693,  0.3892,  0.8007, -0.7500,
          0.9371, -0.9104, -0.9323,  0.6539,  0.2205, -0.9582,  0.5214, -0.9364,
          0.7734, -0.8636, -0.2966,  0.6222, -0.4250,  0.9869, -0.6918, -0.7965,
         -0.9145,  0.5952,  0.9456,  0.8977, -0.3563, -0.1127, -0.7425, -0.8747,
         -0.5934,  0.7949, -

### Réponses

1. Les variables ent0 et ent1 sont les tokens qui représentent la tête et la queue. Les variables sent0, sent1, sent2 sont les tokens pour les autres parties du texte.
2. re_tokens représente le texte complet en tokens. **Il y a aussi des tokens spéciaux qui ont étés introduit par le tokenizeur!"**
3. Dans la méthode tokenize, on encadre la tête et la queue de tokens particuliers "unused" pour pouvoir ensuite les repérer lors du forward.
4. Le Padding permet de remplir le mask d'attention par des 0 lorsqu'on ne prend pas en compte les tokens concernés.**L'operation de padding enchaine à chaque phrase une série de jetons "nuls", jusqu'à avoir une longueur égale à max_length. Ainsi, toutes les phrases traitées auront la même longueur max_length. Cela sert à representer un lot de phrases avec une matrice, ce qui est nécessaire au réseau pour traiter les phrases en parallèle.**
5. L'Attention mask permet de se focaliser sur des tokens, ceux qui sont à 1. On ne prend pas en compte tous les tokens. **Quels tokens ne sont pas pris en compte?**
6. La différence entre la classe BERTEncoder et la classe BERTEntityEncoder est que les méthodes forward et tokenize utilisent les positions de départ de la tête et de la queue pour les repérer.

## NRE Pipeline

Nous allons maintenant programmer une application qui prend en entrée une phrase et donne en sortie deux entités nommées dans la phrase ainsi que la relation entre eux.

Pour ce faire, nous avons besoin :

1. d’un système NER, qui reconnaît les entités nommées dans la phrase
2. d’un système NRE, comme celui que nous avons utilisé jusqu’à là

#### Exercice 4

En vous aident avec la documentation de HuggingFace, instanciez une [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipeline#the-pipeline-abstraction) `ner_pipeline` pour la reconnaissance d'entités nommées, avec le modèle et le tokeniseur pré-entraînes [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Vous pouvez tester la bonne réussite de l’exercice avec le code ci-dessous.

**Indice** : la solution consiste en deux lignes de code : l'une pour importer la classe pipeline, l'autre pour instancier la bonne pipeline.

In [11]:
# Write your code here
from transformers import pipeline
pipe = pipeline("ner")
def ner_pipeline(text):
  
  return pipe(text)

In [12]:
# Apply your ner_pipeline to some sentences to see how it works,
# Then you can test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
print(ner_pipeline(text))
try:
  assert ner_pipeline(text) == [
                                {'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 7, 'score': 0.8588601350784302, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 8, 'score': 0.8766007423400879, 'word': '##el'},
                                {'entity': 'I-PER', 'index': 9, 'score': 0.8375428915023804, 'word': 'D'},
                                {'entity': 'I-PER', 'index': 10, 'score': 0.4897182583808899, 'word': '##ú'},
                                {'entity': 'I-PER', 'index': 11, 'score': 0.901253342628479, 'word': '##in'},
                                {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
                                {'entity': 'I-PER', 'index': 13, 'score': 0.8884768486022949, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 14, 'score': 0.7004077434539795, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 15, 'score': 0.8791088461875916, 'word': '##ele'},
                                {'entity': 'I-PER', 'index': 16, 'score': 0.9720047116279602, 'word': 'Fi'},
                                {'entity': 'I-PER', 'index': 17, 'score': 0.697806715965271, 'word': '##th'},
                                {'entity': 'I-PER', 'index': 18, 'score': 0.697088360786438, 'word': '##rich'},
                                {'entity': 'I-PER', 'index': 26, 'score': 0.9203088283538818, 'word': 'Á'},
                                {'entity': 'I-PER', 'index': 27, 'score': 0.9416706562042236, 'word': '##ed'},
                                {'entity': 'I-PER', 'index': 28, 'score': 0.9702795147895813, 'word': 'U'},
                                {'entity': 'I-PER', 'index': 29, 'score': 0.8428962230682373, 'word': '##ari'},
                                {'entity': 'I-PER', 'index': 31, 'score': 0.7077912092208862, 'word': '##ach'}
                              ]
  print("Good job!")
except AssertionError:
  print("Something might be wrong with your pipeline.") 

[{'word': 'M', 'score': 0.982085645198822, 'entity': 'I-PER', 'index': 6}, {'word': '##á', 'score': 0.8588606715202332, 'entity': 'I-PER', 'index': 7}, {'word': '##el', 'score': 0.876600980758667, 'entity': 'I-PER', 'index': 8}, {'word': 'D', 'score': 0.8375431299209595, 'entity': 'I-PER', 'index': 9}, {'word': '##ú', 'score': 0.4897184371948242, 'entity': 'I-PER', 'index': 10}, {'word': '##in', 'score': 0.9012535810470581, 'entity': 'I-PER', 'index': 11}, {'word': 'mac', 'score': 0.5821399688720703, 'entity': 'I-PER', 'index': 12}, {'word': 'M', 'score': 0.8884767889976501, 'entity': 'I-PER', 'index': 13}, {'word': '##á', 'score': 0.7004088759422302, 'entity': 'I-PER', 'index': 14}, {'word': '##ele', 'score': 0.8791091442108154, 'entity': 'I-PER', 'index': 15}, {'word': 'Fi', 'score': 0.9720048308372498, 'entity': 'I-PER', 'index': 16}, {'word': '##th', 'score': 0.6978078484535217, 'entity': 'I-PER', 'index': 17}, {'word': '##rich', 'score': 0.6970881223678589, 'entity': 'I-PER', 'ind

On obtient des résultats proches du tableau affiché.

#### Exercice 5


Nous pouvons finalement développer une pipeline NRE reposante sur notre modèle de NRE et notre pipeline NER. 

Écrivez ci-dessous une classe `NREPipeline` équipée (entre autres) d'une méthode `__call__(self, text)` qui prend un texte en entrée et effectue les opérations suivantes :

- elle reconnaît les entités dans le texte
    - retenir seulement les deux entités auxquelles la pipeline NER associe la probabilité la plus élevée, écarter les autres (si presentes)
    - si la pipeline NER ne reconnaît qu’une seule entités dans le texte, `__call__(self, text)` retourne None (voir le test en bas) car il n’y a aucune relation à prédire
- elle donne en sortie une liste en format `[e1, e2, rel, p]` où :
    - `e1` est la première entité reconnue dans le texte (entre les deux plus probables, la première qui apparaître dans le texte)
    - `e2` est la deuxième entité  reconnue dans le texte (entre les deux plus probables, la deuxième qui apparaître dans le texte)
    - `rel` est la relation qu’il y a entre `e1` et `e2`
    - `p` est la probabilité associée à la relation `rel` par le modèle de NRE

Pour vérifier la bonne qualité de votre classe `NREPipeline`, utilisez l’extrait de code ci-dessous. Le résultat devrait ressabler à celui-ci :

````
Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['I', 'World Heritage Site', 'heritage designation', 0.9991846680641174]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'location', 0.9127373099327087]
------------------------------
````

In [23]:
# Write your pipeline in this cell. The fun begins here, because we are coding a whole Python class from scratch !
class NREPipeline(object):
  def __init__(self, ner_pipeline, nre_model):
    # ...
    self.ner_pipeline = ner_pipeline
    self.nre_model = nre_model
  # ...

  def __call__(self, text):
   # ...
   ner = ner_pipeline(text)
   grouped_entities = pipe.group_entities(ner)
   if len(grouped_entities) < 2:
     return None
   max1 = 0
   max2 = 0
   entity1 =""
   for entity in grouped_entities:
     if entity['score'] > max1 and entity['score'] > max2:
       max2 = max1
       max1 = entity['score']
       entity2 = entity1
       entity1 = entity['word']
     elif entity['score'] > max2:
       max2 = entity['score']
       entity2 = entity['word']

   item = to_input_format(text, entity1, entity2)

   infer = model.infer(item)

    
   return [entity1, entity2, infer[0], infer[1]]


   #nre_model()


In [24]:
# Test your code with this snippet

sequences = [
             'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', # Easy sentence
             'He was the son of Máel Dúin', # There is only one entity in this sentence, therefore our pipeline should return None
             'Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .', # Ōda is tokenized by the NER tokenizer as a "[UNK]" and it is detected as an entitity. For simplicity, you can discard [UNK] entities as if they were not detected. Here another difficulty arises from the fact that the different tokens in which Iwami Ginzan is decomposed are (wrongly) classified as entities belonging to different classes. In this case, we can consider only the first token "I" as standalone entity, for simplicity.  
             'It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .', # the NER system can not recognise any entity here, therefore the pipeline should return None
             "Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church ." # This sentence is not trivial because the 's has to be managed properly

]

nre = NREPipeline(ner_pipeline, model)
for n, sequence in enumerate(sequences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Áed Uari', 'Máel Dúin mac Máele Fithrich', 'father', 0.9711235761642456]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['World Heritage Site', '[UNK]', 'heritage designation', 0.9965190887451172]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'located in the administrative territorial enti

Les exemple marchent bien.

La relation prédite pour la dernière phrase est 'located in the administrative territorial entity' au lieu de 'location'. Cependant cela reste dans le thème de la localisation/position.

On voit aussi que la reconstitution des entités nommées n'est pas parfaite car les tokens très peu présents sont retirés, il y a donc un trou entre les index des tokens et on ne peut donc pas rassembler toute l'entité nommée.

## Application de le pipeline

#### Exercice 6

1. Appliquez la pipeline NRE aux phrases contenues dans le fichier _sentences.txt_, qui ont été extraites à partir de Wikipedia et Wikidata. Donnez un avis qualitatif sur sa performance, les problèmes rencontrés ainsi que des idées pour améliorations des résultats (e.g. entraînement sur des données différentes, modèle différent, etc.).

2. Appliquez la pipeline NRE aux premières 50 phrases contenues dans le fichier _Sentences_AllAgree.txt_ que vous avez utilisé pour le TP BERT, qui parlent d’événements financiers. Remarquez-vous des différences en terme de performances par rapport à la question 2 ? Pourquoi ? Commentez…

In [27]:
!ls ..

OpenNRE  Sentences_AllAgree.txt  sentences.txt


In [28]:
# Write your code here and then write your detailed answers below. Do not hesitate to use more code cells if needed
sentences = open('../sentences.txt', 'r')
for n, sequence in enumerate(sentences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: The Willard and Josephine Hubbard House was individually listed on the National Register of Historic Places in 2016 .

System out:  ['Josephine Hubbard', 'Willard', 'spouse', 0.8584063649177551]
------------------------------
Sentence 1: His station commander , Group Captain Claude Hilton Keith , found a letter among the missing airman 's personal possessions .

System out:  None
------------------------------
Sentence 2: One was Quintus Caecilius Metellus Creticus , who was praetor in 74 BC and consul in 69 BC .

System out:  None
------------------------------
Sentence 3: April 2009 In addition to musical acts , the label recorded beat poets Lawrence Ferlinghetti and Allen Ginsberg and comic Lenny Bruce .

System out:  ['Lenny Bruce', 'Allen Ginsberg', 'sibling', 0.6458709239959717]
------------------------------
Sentence 4: Walter Neusel ( November 25 , 1907 – October 3 , 1964 ) was a German heavyweight boxer .

System out:  ['Walter Neusel', 'German', 'country of citize

1. Les prédictions sont souvent correctes et il y a souvent des score élevés autour de 90%. Certaines relations sont mal prédites comme par exemple dans la phrase 3 où les 2 entités nommées sont juste 2 poètes mais la relation prédite est "frère et soeur".


Des idées d'amélioration seraient d'entraîner le modèle sur d'autres données où on a déjà rencontré ces entités nommées ou ces relations, mais cela semble compliqué.

Seulement 80 relations sont possibles avec ce modèle, on pourrait utiliser un modèles avec plus de relations. 

Nous avons vu que les entités nommées sont parfois incomplètes. On pourrait travailler sur une meilleure reconstitution des entités nommées.


In [30]:
sentences = open('../Sentences_AllAgree.txt', 'r', encoding='iso-8859-1')
for n, sequence in enumerate(sentences.readlines()[0:50]):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral

System out:  ['Russia', 'Gran', 'owned by', 0.9595130681991577]
------------------------------
Sentence 1: For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive

System out:  ['Componenta', 'EU', 'applies to jurisdiction', 0.4356622099876404]
------------------------------
Sentence 2: In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .@positive

System out:  ['EU', 'EU', 'member of', 0.6252670884132385]
------------------------------
Sentence 3: Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .@positive

System out:  ['EU', 'EU', 'applies to jurisdi

2. Les scores des relations sont souvent plus faibles.
Cela s'explique car il s'agit de phrases d'un thème spécifique : la finance. Il faudrait donc un dataset d'entrainement plus adapté, avec un ensemble de ressources de finance.